Version: 0.0.2  Updated date: 07/05/2024
Conda Environment : py-snowpark_df_ml_fs-1.15.0_v1

# Getting Started with Snowflake Feature Store -  Customer Segmentation

## Setup Database Environment

This Notebook is used to teardown the environment created for the QuickStart when you no longer need it.

In [1]:
%load_ext autoreload
%autoreload 2

#### Notebook Packages

In [2]:
# Python packages
import os
from os import listdir
from os.path import isfile, join
import time
import json
import datetime


# SNOWFLAKE
# Snowpark
from snowflake.snowpark import Session, DataFrame, Window, WindowSpec
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.version import VERSION
from snowflake.ml.utils import connection_params

from useful_fns import run_sql

/opt/miniconda3/envs/py-snowpark_df_ml_fs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup Snowflake connection and database parameters

Change the settings below if you want to if need to apply to your Snowflake Account.

E.g. if you need to use a different role with ACCOUNTADMIN privileges to setup the environment

In [3]:
# Roles
aa_role = 'ACCOUNTADMIN'        # Either a ACCOUNTADMIN, or another role that has been granted ACCOUNTADMIN privileges
fs_qs_role = 'FS_QS_ROLE'       # The Data-Scientist role that will create an have permissions over the data, Feature-Store and Model-Registry

# Database
scale_factor               = 'SF0001'  # TPCXAI data comes in a number of Scale Factors.  For this quickstart we are using the lowest Scale Factor
tpcxai_database_base       = f'TPCXAI_{scale_factor}_QUICKSTART' # The Database we will create to contain the base static data
tpcxai_database_inc        = f'{tpcxai_database_base}_INC' # The Database we will create to contain the pseudo 'Live' incrementing data
databases = [tpcxai_database_base,tpcxai_database_inc]

# Schemas
tpcxai_config_schema       = 'CONFIG'   # This Config Schema containing database artifacts to manage the Quickstart databases
tpcxai_training_schema     = 'TRAINING' # The Training (Development) schema
tpcxai_scoring_schema      = 'SCORING'  # The Scoring (Test) schema
tpcxai_serving_schema      = 'SERVING'  # The Serving (Production) schema
schemas = [tpcxai_training_schema, tpcxai_scoring_schema, tpcxai_serving_schema,]
fq_schemas = []
for d in databases:
    for s in schemas:
        fq_schemas.append(f'''{d}.{s}''')        

# Warehouse
tpcxai_warehouse = f'TPCXAI_{scale_factor}_QUICKSTART_WH'  # The name of the Warehouse we will use for any Quickstart processing
initial_wh_size = 'XSMALL' 

In [4]:
# Create Snowflake Session object
with open('connection.json', 'r') as f:
    connection_parameters = json.load(f)
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True
snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

You might have more than one threads sharing the Session object trying to update sql_simplifier_enabled. Updating this while other tasks are running can potentially cause unexpected behavior. Please update the session configuration before starting the threads.


In [5]:
# Current Environment Details
print('\nConnection Established with the following parameters:')
print(f'User                        : {session.get_current_user()}')
print(f'Role                        : {session.get_current_role()}')
print(f'Database                    : {session.get_current_database()}')
print(f'Schema                      : {session.get_current_schema()}')
print(f'Warehouse                   : {session.get_current_warehouse()}')
print(f'Snowpark for Python version : {snowpark_version[0]}.{snowpark_version[1]}.{snowpark_version[2]} \n')


Connection Established with the following parameters:
User                        : "JARCHEN"
Role                        : "ACCOUNTADMIN"
Database                    : None
Schema                      : None
Warehouse                   : "AICOLLEGE"
Snowpark for Python version : 1.38.0 



## CLEAN UP

In [6]:
run_sql(f'''use role {aa_role}''', session)

use role ACCOUNTADMIN 
 [Row(status='Statement executed successfully.')] 



{'use role ACCOUNTADMIN': [Row(status='Statement executed successfully.')]}

In [7]:
run_sql(f'''drop database {tpcxai_database_base}''', session)

drop database TPCXAI_SF0001_QUICKSTART 
 [Row(status='TPCXAI_SF0001_QUICKSTART successfully dropped.')] 



{'drop database TPCXAI_SF0001_QUICKSTART': [Row(status='TPCXAI_SF0001_QUICKSTART successfully dropped.')]}

In [8]:
run_sql(f'''drop database {tpcxai_database_inc}''', session)

drop database TPCXAI_SF0001_QUICKSTART_INC 
 [Row(status='TPCXAI_SF0001_QUICKSTART_INC successfully dropped.')] 



{'drop database TPCXAI_SF0001_QUICKSTART_INC': [Row(status='TPCXAI_SF0001_QUICKSTART_INC successfully dropped.')]}

In [9]:
run_sql(f'''drop warehouse {tpcxai_warehouse}''', session)

drop warehouse TPCXAI_SF0001_QUICKSTART_WH 
 [Row(status='TPCXAI_SF0001_QUICKSTART_WH successfully dropped.')] 



{'drop warehouse TPCXAI_SF0001_QUICKSTART_WH': [Row(status='TPCXAI_SF0001_QUICKSTART_WH successfully dropped.')]}

In [10]:
session.close()

In [11]:
from datetime import datetime
from zoneinfo import ZoneInfo
formatted_time = datetime.now(ZoneInfo("Australia/Melbourne")).strftime("%A, %B %d, %Y %I:%M:%S %p %Z")

print(f"The last run time in Melbourne is: {formatted_time}")

The last run time in Melbourne is: Monday, September 08, 2025 08:58:38 PM AEST
